# Delete duplicate rows on a table

## Description

We received a database from our junior developer. Everything works fine and the application is doing well. After a few weeks our senior developer for DataWareHouse asked us for some help. He imported duplicated data from our application. Now it´s our turn to solve the problem.

## Requirements

In this example we are going to work with the AdventureWorks database every SQL-Professional knows.

## Preparation

We need to prepare a special table with duplicated data.


In [33]:
-- Create database with duplicates

-- Use AdventureWorks database
USE AdventureWorks2017;

-- Drop table if exists
DROP TABLE IF EXISTS Person.Person_Copy;

-- create new table
CREATE TABLE Person.Person_Copy
(
    [BusinessEntityID] [int] NOT NULL,
    [FirstName] [dbo].[Name] NOT NULL,
    [LastName] [dbo].[Name] NOT NULL
) ON [PRIMARY]

-- fill table with data
INSERT INTO Person.Person_Copy VALUES (1, 'Bob', 'Smith')
INSERT INTO Person.Person_Copy VALUES (2, 'Dave', 'Jones')
INSERT INTO Person.Person_Copy VALUES (3, 'Karen', 'White')
INSERT INTO Person.Person_Copy VALUES (1, 'Bob', 'Smith')

Now lets select out testdata to confirm that there is a duplicate row.

In [35]:
-- select created data

-- Use AdventureWorks database
USE AdventureWorks2017;

-- select all entries
SELECT * FROM Person.Person_Copy;

## Solution

### How to find duplicate records in a table

First we have to find the records which are duplicated

In [50]:
-- select duplicated rows
USE AdventureWorks2017;

SELECT 
    BusinessEntityID,
    FirstName,
    LastName,
    COUNT(*) occurrences
FROM Person.Person_Copy
GROUP BY 
    BusinessEntityID, 
    FirstName,
    LastName
HAVING
    COUNT(*) > 1;

How it works

1. The ``GROUP BY`` clause groups the rows into groups by values in ``BusinessEntityID``, ``FirstName`` and ``LastName`` columns.
2. The ``COUNT()`` function returns the number of occurrences of each group (BusinessEntityID, FirstName, LastName).
3. The ``HAVING`` clause keeps only duplicate groups, which are groups that have more than one occurrence.

### Remove duplicates

We found our duplicated record and now we have to remove it.

Let´s select all duplicates.

In [51]:
-- select all duplicates

USE AdventureWorks2017;

SELECT * FROM Person.Person_Copy
WHERE BusinessEntityID = 1
AND FirstName = 'Bob'
AND LastName = 'Smith';

Now we can remove the first result record with ``TOP`` and select the persons again. To control we select the occurrences for our record.

In [52]:
-- delete the first entry

USE AdventureWorks2017;

DELETE TOP(1) FROM Person.Person_Copy
WHERE BusinessEntityID = 1
AND FirstName = 'Bob'
AND LastName = 'Smith';

SELECT * FROM Person.Person_Copy;

SELECT 
    BusinessEntityID,
    FirstName,
    LastName,
    COUNT(*) occurrences
FROM
    Person.Person_Copy
WHERE 
    BusinessEntityID = 1
    AND FirstName = 'Bob'
    AND LastName = 'Smith'
GROUP BY 
    BusinessEntityID, 
    FirstName,
    LastName

When you find more than two duplicates there is another way to remove all duplicates with one statement. The statement above only removes one duplicate.

In [53]:
-- Create table with more than two duplicates

-- Use AdventureWorks database
USE AdventureWorks2017;

-- Drop table if exists
DROP TABLE IF EXISTS Person.Person_Copy;

-- create new table
CREATE TABLE Person.Person_Copy
(
    [BusinessEntityID] [int] NOT NULL,
    [FirstName] [dbo].[Name] NOT NULL,
    [LastName] [dbo].[Name] NOT NULL
) ON [PRIMARY]

-- fill table with data
INSERT INTO Person.Person_Copy VALUES (2, 'Dave', 'Jones')
INSERT INTO Person.Person_Copy VALUES (1, 'Bob', 'Smith')
INSERT INTO Person.Person_Copy VALUES (2, 'Dave', 'Jones')
INSERT INTO Person.Person_Copy VALUES (2, 'Dave', 'Jones')
INSERT INTO Person.Person_Copy VALUES (3, 'Karen', 'White')
INSERT INTO Person.Person_Copy VALUES (2, 'Dave', 'Jones')
INSERT INTO Person.Person_Copy VALUES (2, 'Dave', 'Jones')
INSERT INTO Person.Person_Copy VALUES (2, 'Dave', 'Jones')
INSERT INTO Person.Person_Copy VALUES (2, 'Dave', 'Jones')

In [54]:
-- select duplicated row
USE AdventureWorks2017;

SELECT 
    BusinessEntityID,
    FirstName,
    LastName,
    COUNT(*) occurrences
FROM Person.Person_Copy
GROUP BY 
    BusinessEntityID, 
    FirstName,
    LastName
HAVING
    COUNT(*) > 1;

In [59]:
-- select all duplicates

USE AdventureWorks2017;

SELECT * FROM Person.Person_Copy
WHERE BusinessEntityID = 2
AND FirstName = 'Dave'
AND LastName = 'Jones';

In [57]:
-- remove all duplicates

USE AdventureWorks2017;

DELETE TOP(SELECT COUNT(*) -1 FROM Person.Person_Copy WHERE BusinessEntityID = 2) FROM Person.Person_Copy
WHERE BusinessEntityID = 2
AND FirstName = 'Dave'
AND LastName = 'Jones';

SELECT * FROM Person.Person_Copy;

SELECT 
    BusinessEntityID,
    FirstName,
    LastName,
    COUNT(*) occurrences
FROM
    Person.Person_Copy
WHERE 
    BusinessEntityID = 2
    AND FirstName = 'Dave'
    AND LastName = 'Jones'
GROUP BY 
    BusinessEntityID, 
    FirstName,
    LastName

In [58]:
-- cleanup

USE AdventureWorks2017;

DROP TABLE Person.Person_Copy;